## Import

In [2]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
%watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

# %matplotlib inline
# %matplotlib widget
# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np

import pandas as pd
import cv2
from glob import glob
import re
import yaml

np.set_printoptions(
    edgeitems=3, infstr='inf',
    linewidth=75, nanstr='nan', precision=6,
    suppress=True, threshold=100, formatter=None)

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x, tag='main', debug=False):
    def __request_text(url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            raise RuntimeError(url)
    try:
        x = x.strip()
        if x[0] == '/' or x[1] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = __request_text(x)
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                x = 'https://' + '/'.join(mod[:3]) + f'/{tag}/' + '/'.join(mod[-3:])
                x = __request_text(x)
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                x = 'https://' + '/'.join(mod[:3]) + f'/raw/{tag}/' + '/'.join(mod[3:])
                x = __request_text(x)
        if debug:
            return x
        else:
            exec(x, globals())
    except Exception as err:
        # sys.stderr.write(f'request {x} : {err}')
       pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


numpy  : 1.24.4
sklearn: 1.0.2
pandas : 1.4.1

ipywidgets: 7.7.1
cv2       : 4.8.1
PIL       : 9.0.1
matplotlib: 3.5.1
plotly    : 5.18.0
netron    : not installed

torch      : 1.12.0.dev20220327+cu113
torchvision: 0.13.0.dev20220327+cu113
torchaudio : 0.12.0.dev20220327+cu113



In [3]:
def  set_rng_seed(x):
    try:
        random.seed(x)
        np.random.seed(x)
        torch.manual_seed(x)
    except: 
        pass
    
import yaml

## Global 

In [4]:
TOP_DIR = '/jupyter/hzcsbet/gamebet'
DATASET_DIR = f'{TOP_DIR}/datasets/SoccerNet/tracking'
CHECKPOINTS_DIR = f'{TOP_DIR}/checkpoints'

labels = ['ball', 'player', 'referee', 'goalkeepers']
label_dict = {'ball': 0, 'player': 1, 'referee': 2, 'goalkeeper': 3, 'goalkeepers': 3}

In [6]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
# model = YOLO('yolov8n.pt')          ### Pre-trained weights

model = YOLO('runs/detect/train4/weights/best.pt')          ### weights from trained model

images = []
# Open the video file
video_path = f"{TOP_DIR}/datasets/0bfacc_5.mp4"
model.predict(source=video_path, save=True)

# cap = cv2.VideoCapture(video_path)
# 
# # Loop through the video frames
# while cap.isOpened():
#     # Read a frame from the video
#     success, frame = cap.read()
# 
#     if success:
#         # Run YOLOv8 tracking on the frame, persisting tracks between frames
#         # results = model.track(frame, persist=True, show=True, tracker="bytetrack.yaml")
#         results = model.predict(
#             source=frame,
#             mode="predict",
#             save=False,
#             # device="cpu"
#         )
#         print(dir(results))
#         break
#         # Visualize the results on the frame
#         # annotated_frame = results[0].plot()
# 
#         # Display the annotated frame
#         # cv2.imshow("YOLOv8 Tracking", annotated_frame)
#         # images.append(annotated_frame)
# 
#     else:
#         # Break the loop if the end of the video is reached
#         break

KeyboardInterrupt: 

In [ ]:
raise

## Data Format Convert

### Data Info

In [5]:
import os

yolo_base = f'{TOP_DIR}/datasets/yolov8'

#(1) image file path
yolo_train_img_dir = f'{yolo_base}/images/train'
yolo_valid_img_dir = f'{yolo_base}/images/valid'

#(2) label file path
yolo_train_label_dir = f'{yolo_base}/labels/train'
yolo_valid_label_dir = f'{yolo_base}/labels/valid'

#(3) config file path
yaml_file = f'{yolo_base}/data.yaml'

os.makedirs(yolo_train_img_dir, exist_ok=True)
os.makedirs(yolo_valid_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True)
os.makedirs(yolo_valid_label_dir, exist_ok=True)

In [6]:
snmot_dirs = {}
snmot_dirs['test'] = sorted(glob(f'{DATASET_DIR}/test/SNMOT*'))
snmot_dirs['train'] = sorted(glob(f'{DATASET_DIR}/train/SNMOT*'))
t1 = snmot_dirs['test'][0]
print(t1)
!ls $t1

/jupyter/hzcsbet/gamebet/datasets/SoccerNet/tracking/test/SNMOT-116
det  gameinfo.ini  gt  img1  seqinfo.ini


In [7]:
!cat $t1/seqinfo.ini

[Sequence]
name=SNMOT-116
imDir=img1
frameRate=25
seqLength=750
imWidth=1920
imHeight=1080
imExt=.jpg


In [8]:
!cat $t1/gameinfo.ini

[Sequence]
name=SNMOT-116
gameID=7
actionPosition=975293
actionClass=Corner
visibility=visible
clipStart=969000
gameTimeStart=1 - 16:09
clipStop=999000
gameTimeStop=1 - 16:39
num_tracklets=27
trackletID_1= player team left;4
trackletID_2= player team left;93
trackletID_3= player team right;25
trackletID_4= referee;main
trackletID_5= player team left;A
trackletID_6= player team left;11
trackletID_7= player team right;34
trackletID_8= player team left;44
trackletID_9= player team right;33
trackletID_10= player team right;20
trackletID_11= player team right;10
trackletID_12= player team right;31
trackletID_13= player team right;14
trackletID_14= player team left;33
trackletID_15= player team left;8
trackletID_16= player team right;36
trackletID_17= player team left;36
trackletID_18= player team right;8
trackletID_19= goalkeepers team left;1
trackletID_20= ball;1
trackletID_21= player team right;B
trackletID_22= player team right;E
trackletID_23= player team left;C
trackletID_24= player te

In [9]:
!head -n 10 $t1/gt/gt.txt 

1,1,136,520,51,135,1,-1,-1,-1
2,1,138,521,50,134,1,-1,-1,-1
3,1,140,521,48,133,1,-1,-1,-1
4,1,142,521,47,132,1,-1,-1,-1
5,1,143,522,46,131,1,-1,-1,-1
6,1,146,523,44,129,1,-1,-1,-1
7,1,146,522,45,129,1,-1,-1,-1
8,1,147,522,45,130,1,-1,-1,-1
9,1,148,521,46,130,1,-1,-1,-1
10,1,149,521,47,131,1,-1,-1,-1


In [10]:
!tail -n 10 $t1/gt/gt.txt 

741,27,148,396,48,113,1,-1,-1,-1
742,27,140,397,49,112,1,-1,-1,-1
743,27,130,399,51,112,1,-1,-1,-1
744,27,125,400,50,112,1,-1,-1,-1
745,27,118,401,48,113,1,-1,-1,-1
746,27,110,403,46,113,1,-1,-1,-1
747,27,103,404,44,113,1,-1,-1,-1
748,27,94,405,42,114,1,-1,-1,-1
749,27,89,406,42,114,1,-1,-1,-1
750,27,80,409,42,114,1,-1,-1,-1


In [11]:
!cat $t1/seqinfo.ini

[Sequence]
name=SNMOT-116
imDir=img1
frameRate=25
seqLength=750
imWidth=1920
imHeight=1080
imExt=.jpg


### Parse Info

In [12]:
# convert from x,y,w,h to yolo format
def get_yolo_format_bbox(img_w, img_h, box):
    w = int(box[2])
    h = int(box[3])
    xc = int(box[0]) + int(np.round(w/2))
    yc = int(box[1]) + int(np.round(h/2))
    box = [xc/img_w, yc/img_h, w/img_w, h/img_h]
    box = [f"{i:.4g}" for i in box]
    return box
    
# get SoccerNet label info 
def get_game_info(info):
    results = {}
    for line in open(info):
        m = re.match('trackletID_(\d+)= (\S*).*', line.replace(';', ' '))
        if m:
            if m.group(2) not in label_dict:
                continue 
            results[m.group(1)] = m.group(2)
    return results

def get_seq_info(info):
    results = {}
    for line in open(info):
        if line.startswith('name'):
            results['name'] = line.split('=')[1].strip()
        elif line.startswith('imDir'):
            results['img_dir'] = line.split('=')[1].strip()
        elif line.startswith('imWidth'):
            results['img_w'] = int(line.split('=')[1].strip())
        elif line.startswith('imHeight'):
            results['img_h'] = int(line.split('=')[1].strip())
        elif line.startswith('imExt'):
            results['img_ext'] = line.split('=')[1].strip()
    return results

In [13]:
seq_info = get_seq_info(f'{t1}/seqinfo.ini')
seq_info

{'name': 'SNMOT-116',
 'img_dir': 'img1',
 'img_w': 1920,
 'img_h': 1080,
 'img_ext': '.jpg'}

In [14]:
game_info = get_game_info(f'{t1}/gameinfo.ini')
game_info

{'1': 'player',
 '2': 'player',
 '3': 'player',
 '4': 'referee',
 '5': 'player',
 '6': 'player',
 '7': 'player',
 '8': 'player',
 '9': 'player',
 '10': 'player',
 '11': 'player',
 '12': 'player',
 '13': 'player',
 '14': 'player',
 '15': 'player',
 '16': 'player',
 '17': 'player',
 '18': 'player',
 '19': 'goalkeepers',
 '20': 'ball',
 '21': 'player',
 '22': 'player',
 '23': 'player',
 '24': 'player',
 '25': 'referee',
 '26': 'player',
 '27': 'player'}

In [ ]:
results = []
for phase, dirs in snmot_dirs.items():
    if len(dirs) == 0:
        continue
    for mot in dirs:
        seq_info = get_seq_info(f'{mot}/seqinfo.ini')
        game_info = get_game_info(f'{mot}/gameinfo.ini')
        result = {}
        for line in open(f'{mot}/gt/gt.txt'):
            imgid, labid, *box = line.split(',')[:6]
            if labid in game_info:
                imgid = f'{int(imgid):06}'
                if imgid not in result:
                    result[imgid] = []
                box = get_yolo_format_bbox(seq_info['img_w'], seq_info['img_h'], box)
                result[imgid].append([label_dict[game_info[labid]], *box])
        seq_info['data'] = result
        seq_info['phase'] = phase
        results.append(seq_info)

### Train and Valid Data

In [ ]:
set_rng_seed(888)
for item in results:
    for imgid, data in item['data'].items():
        img_dst_file = f'{item["name"]}_{imgid}{item["img_ext"]}'
        lab_dst_file = f'{item["name"]}_{imgid}.txt'
        img_src_path = f'{DATASET_DIR}/{item["phase"]}/{item["name"]}/{item["img_dir"]}/{imgid}{item["img_ext"]}'
        if not os.path.exists(img_src_path):
            print('Not found: ', img_src_path)
            continue
        if np.random.random(1) > 0.9:
            img_dst_path = f'{yolo_valid_img_dir}/{img_dst_file}'
            lab_dst_path = f'{yolo_valid_label_dir}/{lab_dst_file}'
        else:
            img_dst_path = f'{yolo_train_img_dir}/{img_dst_file}'
            lab_dst_path = f'{yolo_train_label_dir}/{lab_dst_file}'
        if not os.path.exists(img_dst_path):
            os.symlink(img_src_path, img_dst_path)
        with open(lab_dst_path, 'w') as fw:
            for line in data:
                fw.write(f'%d %s %s %s %s\n' % (line[0], line[1], line[2], line[3], line[4]))

In [ ]:
data_yaml = dict(
    train = yolo_train_img_dir,
    val = yolo_valid_img_dir,
    nc = 4,
    names = labels
)

with open(yaml_file, 'w') as fw:
    yaml.dump(data_yaml, fw, default_flow_style=False)

## Train

In [15]:
from ultralytics import YOLO

In [ ]:
model = YOLO(f'{CHECKPOINTS_DIR}/yolov8m.pt')
results = model.train(data=yaml_file, epochs=100, batch=64, imgsz=640, save_dir=CHECKPOINTS_DIR)

Ultralytics YOLOv8.0.229 🚀 Python-3.8.10 torch-1.12.0.dev20220327+cu113 CUDA:0 (Tesla P40, 24452MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=/jupyter/hzcsbet/gamebet/checkpoints/yolov8m.pt, data=/jupyter/hzcsbet/gamebet/datasets/yolov8/data.yaml, epochs=100, time=None, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


Transferred 469/475 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train6', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /data/hzcsai_com/hzcsbet/gamebet/datasets/yolov8/labels/train.cache... 33056 images, 0 backgrounds, 0 corrupt: 100% 33056/33056 [00:00<?, ?it/s]

train: WARNING ⚠️ /data/hzcsai_com/hzcsbet/gamebet/datasets/yolov8/images/train/SNMOT-194_000585.jpg: 1 duplicate labels removed


...

  0% 0/517 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
      4/100      24.5G       1.21     0.5755     0.9027        858        640: 100% 517/517 [25:49<00:00,  3.00s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 29/29 [00:47<00:00,  1.65s/it]


                   all       3694      56328      0.851       0.76      0.796      0.494

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0% 0/517 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
      5/100      24.2G      1.158     0.5434     0.8925        716        640: 100% 517/517 [25:44<00:00,  2.99s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 29/29 [00:49<00:00,  1.71s/it]


                   all       3694      56328      0.884       0.77      0.811      0.526

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0% 0/517 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
      6/100      24.2G       1.13     0.5277     0.8863       1699        640:  20% 102/517 [05:04<20:37,  2.98s/it]

## Test

In [ ]:
import numpy as np
np.__version__

## References

- [BoT-SORT: Robust Associations Multi-Pedestrian Tracking][5]
- [ByteTrack: Multi-Object Tracking by Associating Every Detection Box][1]
- [SoccerNet - Tracking][2]
- [Cool experiments at the intersection of Computer Vision and Sports][3]
- [Football players tracking — identifying players’ team based on their jersey colors using OpenCV][4]
- https://readmedium.com/chess-rolls-or-basketball-lets-create-a-custom-object-detection-model-ef53028eac7d
- https://medium.com/@amirhossein477/how-to-obtain-the-birds-eye-view-of-a-soccer-game-regardless-of-camera-angle-changes-90d627acd522
- https://medium.com/@amritangshu.mukherjee/tracking-football-players-with-yolov5-bytetrack-efa317c9aaa4
- https://www.jlburkhead.com

[5]: https://arxiv.org/pdf/2
[4]: https://towardsdatascience.com/football-players-tracking-identifying-players-team-based-on-their-jersey-colors-using-opencv-7eed1b8a1095
[3]: https://github.com/SkalskiP/sports
[2]: https://github.com/SoccerNet/sn-tracking
[1]: https://arxiv.org/pdf/2110.06864.pdf